- In this we will see everything related to `Filed data types`

### Common Data-type

- Binry datatype

![Binary datatypes](../img-src/Storing_image_as_binry.png)

In [5]:
import base64

In [6]:
from elasticsearch import Elasticsearch
import json
from pprint import pprint

-   create an index

In [7]:
es= Elasticsearch(hosts="http://localhost:9200/")
pprint(es.info().body)

{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'tGYnSe7ISE2SFo0pu1RcHg',
 'name': '5d9dbd669d35',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-02-13T09:35:20.314882762Z',
             'build_flavor': 'default',
             'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.4.2',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.6.2'}}


In [8]:
es.indices.delete(index="test_index", ignore_unavailable= True)

es.indices.create(index="test_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

- `test_index` created, Now read and image convert into base64 bitecode and then decode It in utf-8 string for storage

In [9]:
image_str_data=""
with open('../img-src/test_image.png', "rb") as imageFD:
    image_binary_data=imageFD.read()
    image_str_data= base64.b64encode(image_binary_data).decode("utf-8")



In [10]:
print(image_str_data[:100])

iVBORw0KGgoAAAANSUhEUgAAAyYAAAGeCAYAAABhMRUDAAAABHNCSVQICAgIfAhkiAAAABl0RVh0U29mdHdhcmUAZ25vbWUtc2Ny


-  insert the image str in test_index

In [11]:
# convert the image data in a json format
doc={
    "image_data": image_str_data
}

# store the doc in test_index

res=es.index(index="test_index", document=doc)



In [12]:
pprint(res.body)

{'_id': 'fl5dF5UBIxIS9hU_YKGT',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [13]:
# see the automatic mapping
infered_mapping=es.indices.get_mapping(index="test_index")["test_index"]["mappings"]

In [14]:
pprint(infered_mapping)

{'properties': {'image_data': {'fields': {'keyword': {'ignore_above': 256,
                                                      'type': 'keyword'}},
                               'type': 'text'}}}


## JSON object storage

# 3.1 Text

In [15]:
es.indices.delete(index="test_index", ignore_unavailable= True)
mapping={
    "properties":{
        "email_body":{"type":"text"}
    }
}
es.indices.create(index="test_index", mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

In [16]:
doc={
    "email_body":"Hello, this is a test email body. regards, verma"
}
res=es.index(index="test_index", document=doc)
pprint(res.body)

{'_id': 'f15dF5UBIxIS9hU_YaH0',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [17]:
es.indices.get_mapping(index="test_index")['test_index']['mappings']

{'properties': {'email_body': {'type': 'text'}}}

# 3.2 Completion

In [18]:
es.indices.delete(index="test_index", ignore_unavailable=True)
mapping={
    "properties":{
        "suggest":{ #compulsory field
            "type":"completion"
        }
    }
}
es.indices.create(index="test_index", mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'})

In [19]:
doc1={
    "suggestion":{
        "input":["Mars", "Planet"] #copulsory field
    }
}
doc2={
    "suggestion":{
        "input":["Moon", "Satellite"] #copulsory field
    }
}

res1=es.index(index="test_index", document=doc1)
res2=es.index(index="test_index", document=doc1)

In [20]:
pprint(res1.body)
pprint(res2.body)

{'_id': 'gF5dF5UBIxIS9hU_YqGx',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}
{'_id': 'gV5dF5UBIxIS9hU_YqHW',
 '_index': 'test_index',
 '_primary_term': 1,
 '_seq_no': 1,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


# 4.1 Geo point

In [21]:
es.indices.delete(index="geo_point_index", ignore_unavailable= True)
mappings={
    "properties":{
        "location":{
            "type":"geo_point"
        }
    }
}
es.indices.create(index="geo_point_index", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_point_index'})

In [27]:
geo_doc={
    "text":"Geopoint as object using GeoJSON object",
    "location":{
        "type":"Point",
        "coordinates":[
            -71.34,
            41.12
        ]
    }
}
res=es.index(index="geo_point_index", document=geo_doc)

In [28]:
pprint(res.body)

{'_id': 'hF57F5UBIxIS9hU_aaFE',
 '_index': 'geo_point_index',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


# 4.2 Geo shape

In [29]:
es.indices.delete(index="geo_shape_index", ignore_unavailable=True)
mappings={
    "properties":{
        "location":{"type":"geo_shape"}
    }
}
es.indices.create(index="geo_shape_index", mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'geo_shape_index'})

In [41]:
doc1={
  "location": {
    "type": "linestring",
    "coordinates": [
      [13.400544, 52.530286],
      [13.402344, 52.532984],
      [13.403123, 52.534567]
    ]
  }
}

doc2={
    "location": {
        "type": "polygon",
        "coordinates":[
            [
                [13.400544, 52.530286],
                [13.402344, 52.532984],
                [13.403123, 52.534567],
                [13.400544, 52.530286]  # Closing the polygon by repeating the first coordinate
            ]
        ]
  }
}


In [42]:
res1=es.index(index="geo_shape_index",document=doc1)
res2=es.index(index="geo_shape_index",document=doc2)

In [43]:
pprint(res1.body)
pprint(res2.body)

{'_id': 'i16NF5UBIxIS9hU_2aEy',
 '_index': 'geo_shape_index',
 '_primary_term': 1,
 '_seq_no': 2,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}
{'_id': 'jF6NF5UBIxIS9hU_2aFO',
 '_index': 'geo_shape_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


# 4.3 Point (cartesian)

In [44]:
es.indices.delete(index="point_index", ignore_unavailable=True)
mapping={
    "properties":{
        "position":{
            "type":"point"
        }
    }
}
es.indices.create(index="point_index", mappings=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'point_index'})

In [45]:
doc={
    "position":{
        "type":"Point",
        "coordinates":[
            -324.45,
            324.232
        ]
    }
}

In [46]:
res=es.index(index="point_index", document=doc)

In [47]:
pprint(res.body)

{'_id': 'jV6SF5UBIxIS9hU_naH-',
 '_index': 'point_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}
